In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("data_transformed_2.csv")
df.head()

,time,open,high,low,close,intraday_change_$,volume,close_change_$,close_change_%,volume_change,volume_change_%,5_min_avg_$,8_min_avg_$,13_min_avg_$,20_min_avg_$,50_min_avg_$,200_min_avg_$,minute_volatility_$,price_change_binary
0,5/17/19 11:52,287.375,287.44,287.270,287.43,0.05,63474,0.05,0.017399,-70537,-52.635231,287.405,287.405000,287.405000,287.405000,287.405000,287.405000,0.170,1
1,5/17/19 11:53,287.430,287.57,287.405,287.51,0.08,116036,0.08,0.027833,52562,82.808709,287.440,287.440000,287.440000,287.440000,287.440000,287.440000,0.165,1
2,5/17/19 11:54,287.500,287.58,287.430,287.58,0.07,60199,0.07,0.024347,-55837,-48.120411,287.475,287.475000,287.475000,287.475000,287.475000,287.475000,0.150,1
3,5/17/19 11:55,287.585,287.62,287.480,287.50,-0.08,88436,-0.08,-0.027818,28237,46.906095,287.480,287.480000,287.480000,287.480000,287.480000,287.480000,0.140,0
4,5/17/19 11:56,287.490,287.50,287.390,287.46,-0.04,130754,-0.04,-0.013913,42318,47.851554,287.480,287.476667,287.476667,287.476667,287.476667,287.476667,0.110,0


### Adding 'Price Category' Column

In [4]:
up_list=[]
down_list=[]
momentum_up={}
momentum_down={}
for index, row in df.iterrows():
    # If 1 and switching from downward trend to upward trend
    if (row["price_change_binary"]==1) and (0 in down_list):
        if str(len(down_list)) in momentum_down.keys():
            # Add count of downward move to dictionary
            momentum_down[str(len(down_list))]-=1
            # Add downward category
            for i in range(len(down_list)+1)[1:]:
                df.loc[index-i, "price_category"]=-len(down_list) 
            # Add new element in upward count list
            up_list.append(1)
            # Clear down_list
            down_list=[]
        else:
            # Add new count with key
            momentum_down[str(len(down_list))]=-1
            # Add downward category
            for i in range(len(down_list)+1)[1:]:
                df.loc[index-i, "price_category"]=-len(down_list)  
            # Clear down_list
            down_list=[]
            # Add new element in upward count list
            up_list.append(1)
    elif row["price_change_binary"]==1:
        up_list.append(1)
    
    # if 0 and switching from upward trend to downward trend 
    if (row["price_change_binary"]==0) and (1 in up_list):
        # Adding count of upward move to dictionary
        if str(len(up_list)) in momentum_up.keys():
            # Add count of upward move to dictionary
            momentum_up[str(len(up_list))]+=1
            # Add upward category
            for i in range(len(up_list)+1)[1:]:
                df.loc[index-i, "price_category"]=len(up_list)
            # Clear up_list
            up_list=[]
        else:
            # Add count of upward move to dictionary
            momentum_up[str(len(up_list))]=1
            # Add upward category
            for i in range(len(up_list)+1)[1:]:
                df.loc[index-i, "price_category"]=len(up_list)
            # Clear up_list
            up_list=[]
        # Creating new element in downward count list
        down_list.append(0)
    # if on downward trend, add another 0
    elif row["price_change_binary"]==0:
        down_list.append(0)

### Momentum
* Keys = Number of up-ticks in a given price swing, Values = Count of given price swing
* Example Reading: {"3":4679"} -> SPY moved upward three minutes in a row before moving downward 4679 times

In [5]:
momentum_up

{'3': 4679,
 '1': 17855,
 '2': 9430,
 '6': 526,
 '4': 2250,
 '5': 1126,
 '7': 229,
 '8': 109,
 '10': 21,
 '9': 42,
 '11': 9,
 '15': 1,
 '12': 2,
 '13': 5,
 '14': 1}

* Example Reading: {"3":4834"} -> SPY moved downward three minutes in a row before moving upward 4834 times

In [6]:
momentum_down

{'3': -4834,
 '2': -9297,
 '4': -2502,
 '1': -17148,
 '5': -1278,
 '6': -601,
 '8': -159,
 '7': -320,
 '9': -72,
 '13': -3,
 '10': -35,
 '17': -2,
 '15': -2,
 '12': -8,
 '11': -21,
 '19': -1,
 '14': -1}

In [7]:
df.head()

,time,open,high,low,close,intraday_change_$,volume,close_change_$,close_change_%,volume_change,volume_change_%,5_min_avg_$,8_min_avg_$,13_min_avg_$,20_min_avg_$,50_min_avg_$,200_min_avg_$,minute_volatility_$,price_change_binary,price_category
0,5/17/19 11:52,287.375,287.44,287.270,287.43,0.05,63474,0.05,0.017399,-70537,-52.635231,287.405,287.405000,287.405000,287.405000,287.405000,287.405000,0.170,1,3.0
1,5/17/19 11:53,287.430,287.57,287.405,287.51,0.08,116036,0.08,0.027833,52562,82.808709,287.440,287.440000,287.440000,287.440000,287.440000,287.440000,0.165,1,3.0
2,5/17/19 11:54,287.500,287.58,287.430,287.58,0.07,60199,0.07,0.024347,-55837,-48.120411,287.475,287.475000,287.475000,287.475000,287.475000,287.475000,0.150,1,3.0
3,5/17/19 11:55,287.585,287.62,287.480,287.50,-0.08,88436,-0.08,-0.027818,28237,46.906095,287.480,287.480000,287.480000,287.480000,287.480000,287.480000,0.140,0,-3.0
4,5/17/19 11:56,287.490,287.50,287.390,287.46,-0.04,130754,-0.04,-0.013913,42318,47.851554,287.480,287.476667,287.476667,287.476667,287.476667,287.476667,0.110,0,-3.0


In [8]:
# Manually editing the data
df.at[147039,"price_category"]=0
df

,time,open,high,low,close,intraday_change_$,volume,close_change_$,close_change_%,volume_change,volume_change_%,5_min_avg_$,8_min_avg_$,13_min_avg_$,20_min_avg_$,50_min_avg_$,200_min_avg_$,minute_volatility_$,price_change_binary,price_category
0,5/17/19 11:52,287.375,287.44,287.2700,287.4300,0.0500,63474,0.0500,0.017399,-70537,-52.635231,287.40500,287.405000,287.405000,287.405000,287.405000,287.405000,0.1700,1,3.0
1,5/17/19 11:53,287.430,287.57,287.4050,287.5100,0.0800,116036,0.0800,0.027833,52562,82.808709,287.44000,287.440000,287.440000,287.440000,287.440000,287.440000,0.1650,1,3.0
2,5/17/19 11:54,287.500,287.58,287.4300,287.5800,0.0700,60199,0.0700,0.024347,-55837,-48.120411,287.47500,287.475000,287.475000,287.475000,287.475000,287.475000,0.1500,1,3.0
3,5/17/19 11:55,287.585,287.62,287.4800,287.5000,-0.0800,88436,-0.0800,-0.027818,28237,46.906095,287.48000,287.480000,287.480000,287.480000,287.480000,287.480000,0.1400,0,-3.0
4,5/17/19 11:56,287.490,287.50,287.3900,287.4600,-0.0400,130754,-0.0400,-0.013913,42318,47.851554,287.48000,287.476667,287.476667,287.476667,287.476667,287.476667,0.1100,0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147035,11/13/20 15:55,357.920,357.96,357.8100,357.8789,-0.0411,492569,-0.0411,-0.011483,102932,26.417409,356.41100,356.460438,356.471977,356.516195,356.648530,357.163052,0.1500,0,-4.0
147036,11/13/20 15:56,357.860,357.95,357.8000,357.8200,-0.0589,309294,-0.0589,-0.016458,-183275,-37.207985,356.46170,356.487537,356.490823,356.532945,356.664624,357.170727,0.1500,0,-4.0
147037,11/13/20 15:57,357.825,358.21,357.8118,358.2050,0.3850,479111,0.3850,0.107596,169817,54.904718,356.49370,356.502538,356.509285,356.543445,356.675824,357.179802,0.3982,1,2.0
147038,11/13/20 15:58,358.205,358.35,358.1400,358.3500,0.1450,718921,0.1450,0.040480,239810,50.053119,356.52570,356.525087,356.528131,356.556445,356.683026,357.189652,0.2100,1,2.0


In [10]:
# Save DataFrame as CSV file
df.to_csv("data_transformed_3.csv", index=False)